# Import Modules/Packages

In [114]:
import folium
import pandas as pd
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import addfips
import requests
import time
import random

# Get SoCal COVID-19 Cases from LATimes Data, Scrape Data, and Make/Save to DF/CSV

LATimes Table

https://www.thepythoncode.com/article/convert-html-tables-into-csv-files-in-python

BS4 files from py4e


In [8]:
url = "https://www.latimes.com/projects/california-coronavirus-cases-tracking-outbreak"

def get_soup(url):
    """Ignore SSL certificate errors"""
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    """Gets Soup"""
    html = urllib.request.urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, 'html.parser')
    return soup

soup = get_soup(url)

#body > article > div:nth-child(8) > section > div.small-table.small-table__socal-map > table
#/html/body/article/div[6]/section/div[2]/table
table = soup.find_all('table')[1]

"""Given a table soup, returns all the headers"""
headers = []
for th in table.find("tr").find_all("th"):
    headers.append(th.text.strip())
    
"""Given a table, returns all its rows"""
rows = []
for tr in table.find_all("tr")[1:]:
    cells = []
    for item in tr:
        cells.append(item.text.strip())
    rows.append(cells)
    
table_name = "SoCal County Data"

df = pd.DataFrame(rows,columns=headers)

df.loc[(df['Cases'] == '1-4'),"Cases"] = 1

df["Cases"] = df["Cases"].apply(pd.to_numeric)

#convert first column of table into list if needed
geolist=[]
for item in rows:
    geolist.append(item[0])

df.to_csv("SoCal-COVID-Data-Raw-Table.csv")

# Using GEOPY to get Geocode and Coordinates

### References & Resources

https://www.youtube.com/watch?v=q_OUHA_zqeM


In [157]:
# from geopy.geocoders import Nominatim
# nom=Nominatim(user_agent="my-application", timeout = 100)
# #nom.geocode("Artesia, Los Angeles, CA")

# df["Full_Location"]=df["Area"]+", "+df["County"]+", California"
# #df.drop(columns=["Location"])
# df=df[["Full_Location","Area","County","Cases"]]

# df["Geolocation"]=df["Full_Location"].apply(nom.geocode)
# df.to_csv("SoCal-COVID-Data-Geopy.csv")

# Create Lat/Long Columns and Save to CSV

In [162]:
# df["Latitude"]=df["Geolocation"].apply(lambda x: x.latitude if x != None else None)
# df["Longitude"]=df["Geolocation"].apply(lambda x: x.longitude if x != None else None)
# df.to_csv("SoCal-COVID-Data-Geopy_FULL.csv")

# Convert DF Lat/Lon NaN to Zeros

In [196]:
# Converted NaN to zeros
# df.loc[(df.Longitude.isnull(), "Longitude")] = 0
# df.loc[(df.Latitude.isnull(), "Latitude")] = 0

Series([], Name: Longitude, dtype: float64)

# Load in previously scraped data, so you don't have to keep scraping


In [48]:

df_full = pd.read_csv("/Users/Dave/DBP0001/SoCal-COVID-Data-Geopy_FULL.csv")
df_full = df_full.drop(columns=["Unnamed: 0"])
df_full

# ##Converted NaN to zeros
df_full.loc[(df_full.Longitude.isnull(), "Longitude")] = 0
df_full.loc[(df_full.Latitude.isnull(), "Latitude")] = 0


# Create Dictionary with Lat, Long, FIPS, and Location Area

In [94]:
###Find FIPS
#https://gis.stackexchange.com/questions/294641/python-code-for-transforming-lat-long-into-fips-codes

###There are duplicate "Area" values, so all of this is needed to ensure duplicates with differing lat/long vals are kept
location_list = df_full['Area'].tolist()
lat_list = df_full['Latitude'].tolist()
long_list = df_full['Longitude'].tolist()
index_list=df_full.index.tolist()

latlong_list=[list(a) for a in zip(lat_list, long_list)]

loc_with_latlong_list_full = []

for i in range(len(location_list)):
    loc_with_latlong_list_pre = []
    loc_with_latlong_list_pre.append(location_list[i])
    loc_with_latlong_list_pre.append(latlong_list[i])
    loc_with_latlong_list_full.append(loc_with_latlong_list_pre)

loc_with_latlong_list_full

loc_dict={}

for i in range(len(index_list)):
    loc_dict[index_list[i]]=loc_with_latlong_list_full[i]
        
loc_dict

{0: ['San Diego', [32.7174209, -117.16277140000001]],
 1: ['Long Beach', [33.7690164, -118.1916048]],
 2: ['Melrose', [34.0536909, -118.2427666]],
 3: ['Glendale', [34.146941600000005, -118.24784709999999]],
 4: ['Santa Clarita', [34.3916641, -118.54258600000001]],
 5: ['Hollywood', [34.0980031, -118.32952320000001]],
 6: ['North Hollywood', [34.1729044, -118.37403710000001]],
 7: ['Torrance', [33.835849200000006, -118.34062879999999]],
 8: ['Chula Vista', [32.6400541, -117.0841955]],
 9: ['Carson', [33.832204299999994, -118.2517547]],
 10: ['West Hollywood', [34.0923014, -118.36928940000001]],
 11: ['Anaheim', [33.8347516, -117.911732]],
 12: ['Irvine', [33.685696899999996, -117.82598190000002]],
 13: ['Moreno Valley', [33.937517, -117.23059440000002]],
 14: ['Other/Unknown', [0.0, 0.0]],
 15: ['Newport Beach', [33.6170092, -117.92944009999998]],
 16: ['Burbank', [34.181648200000005, -118.3258554]],
 17: ['Yucaipa', [34.033625, -117.0430865]],
 18: ['Inglewood', [33.9562003, -118.3531

# Create a FIPS list using FCC Geo API from Dictionary

In [97]:
fips_list=[]
 
for k,v in loc_dict.items():
    #Encode parameters 
    params = urllib.parse.urlencode({'latitude': loc_dict[k][1][0], 'longitude':loc_dict[k][1][1], 'format':'json'})
    #Contruct request URL
    url = 'https://geo.fcc.gov/api/census/block/find?' + params
    #Get response from API
    response = requests.get(url)
    #Parse json in response
    data = response.json()
    #Print FIPS code
    fips_list.append(data['County']['FIPS'])
    #time.sleep(random.uniform(0, 1))
    
fips_list

['06073',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06073',
 '06037',
 '06037',
 '06059',
 '06059',
 '06065',
 None,
 '06059',
 '06037',
 '06071',
 '06037',
 '06037',
 '06037',
 '06073',
 '06037',
 '06037',
 '06059',
 '06065',
 '06037',
 '06059',
 '06059',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06065',
 '06037',
 '06111',
 '06037',
 '06083',
 '06037',
 '06065',
 '06065',
 '06065',
 '06065',
 '06037',
 '06071',
 '06111',
 '06073',
 '06037',
 '06065',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06059',
 '06037',
 '06037',
 '06037',
 '06037',
 '06111',
 '06037',
 '06065',
 '06037',
 '06037',
 '06059',
 '06083',
 '06065',
 '06037',
 '06037',
 '06111',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06083',
 '06037',
 '06073',
 '06037',
 '06071',
 '06037',
 '06037',
 '06073',
 '06037',
 '06073',
 '06037',
 '06059',
 '06037',
 '06073',
 '06037',
 '06071',
 '06037',
 '06037',
 '0

# Remove "None" from FIPS List

In [98]:
for i in range(len(fips_list)):
    if fips_list[i] == None:
        fips_list[i] = '00000'

fips_list

['06073',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06073',
 '06037',
 '06037',
 '06059',
 '06059',
 '06065',
 '00000',
 '06059',
 '06037',
 '06071',
 '06037',
 '06037',
 '06037',
 '06073',
 '06037',
 '06037',
 '06059',
 '06065',
 '06037',
 '06059',
 '06059',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06065',
 '06037',
 '06111',
 '06037',
 '06083',
 '06037',
 '06065',
 '06065',
 '06065',
 '06065',
 '06037',
 '06071',
 '06111',
 '06073',
 '06037',
 '06065',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06059',
 '06037',
 '06037',
 '06037',
 '06037',
 '06111',
 '06037',
 '06065',
 '06037',
 '06037',
 '06059',
 '06083',
 '06065',
 '06037',
 '06037',
 '06111',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06037',
 '06083',
 '06037',
 '06073',
 '06037',
 '06071',
 '06037',
 '06037',
 '06073',
 '06037',
 '06073',
 '06037',
 '06059',
 '06037',
 '06073',
 '06037',
 '06071',
 '06037',
 '06037',


# Send FIPS list to Pandas Dataframe with full data


In [100]:
#loc_dict has ~10 fewer entries than in df because there are duplicate "Area" values in Dataframe --> drop duplicates

df_full["FIPS"]=fips_list
df_full.to_csv("/Users/Dave/DBP0001/SoCal-COVID-Data-Geopy_FULL_withFIPS.csv")
# print(df_full.shape)
# print(len(fips_list))

# print(len(location_list))
# print(len(lat_list))
# print(len(long_list))
# print(len(loc_dict))


# # print(df_full["Area"].value_counts())


# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)
# df_full["Area"].value_counts()


# df_full[df_full["Area"]=="Azusa"]
# df_full[df_full.duplicated(["Area"], keep=False)]


# Export coordinates to geojson

### References & Resources 

https://geoffboeing.com/2015/10/exporting-python-data-geojson/

In [240]:
def df_to_geojson(df, properties, lat='Latitude', lon='Longitude',fips='FIPS'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]},
                  'id':[]}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        feature['id']= row[fips]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols=["Area","Cases"]
geojson_dict = df_to_geojson(df_full, properties=cols)
geojson_str = json.dumps(geojson_dict, indent=2)

output_filename = 'covid-map-dataset.json'
with open(output_filename, 'w') as output_file:
    output_file.write('{}'.format(geojson_str))

# LOAD IN DF_FULL (FROM CSV) & JSON FROM PROJECT TO AVOID ALL CELLS ABOVE

In [242]:
#csv_file = "SoCal-COVID-Data-Geopy_FULL_withFIPS.csv"
#json_file = "covid-map-dataset.json"
#df_full=pd.read_csv(csv_file)

# Plotly Choropleth Map Attempt

### References & Resources

https://github.com/yg2619/Choropleth-Maps-in-Python-Using-Plotly/blob/master/choropleth-map.py

https://github.com/yg2619/Choropleth-Maps-in-Python-Using-Plotly

https://plotly.com/python/mapbox-density-heatmaps/

*PROBLEM: Choropleth shades not filling in*

In [237]:
import plotly
import plotly.graph_objs as go
import json

#df_full is pd dataframe from above cells

with open('/Users/Dave/covid-map-dataset.json') as file:
    socalmap = json.load(file)

fig = go.Figure(go.Choroplethmapbox(geojson=socalmap,locations=df_full.FIPS,z=df_full.Cases,
                                    colorscale='Cividis',zmin=0,zmax=800,
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=8, mapbox_center = {"lat": 34.0522, "lon": -118.2437})

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


# Plotly Express Density Mapbox Attempt

### References & Resources

https://github.com/yg2619/Choropleth-Maps-in-Python-Using-Plotly

https://plotly.com/python/mapbox-density-heatmaps/

https://plotly.com/python/mapbox-county-choropleth/

*PROBLEM: Color bar colors reversed + Color only intensifies when zooming out*

In [236]:
import plotly.express as px

#df_full is pd dataframe from above cells

fig = px.density_mapbox(df_full, lat='Latitude', lon='Longitude', z='Cases', radius=10,
                        center=dict(lat=34.0522, lon=-118.2437), zoom=8,
                        mapbox_style="carto-positron",opacity=1.0)
fig.show()

# OLD CODE ATTEMPTS / NOT WORKING

In [150]:
# ##https://www.youtube.com/watch?v=D35m2CdMhVs

# import plotly.figure_factory as ff

# scope = ['California']

# # list values
# cases = df_full['Cases'].tolist()
# fips = df_full['FIPS'].tolist()

# #have to have the colorscale values at least = to the # of counties
# colorscale = ["#2daa4b", "#80b1d3", "#fdb462", "#b3de69", "#fccde5", "#ff0000", "#cce6f4",
#               "#80b1d3", "#fdb462", "#b3de69", "#fccde5", "#9ba991", "#604d74", "#afcbc7",
#               "#80b1d3", "#fdb462", "#b3de69", "#fccde5", "#f0c800", "#387e75", "#fce7e7",
#               "#8dd3c7", "#ffffb3", "#bebada", "#fb8072", "#00778b", "#1d2d43", "#460606",
#               "#8dd3c7", "#ffffb3", "#bebada", "#fb8072", "#af3205", "#436e6f", "#f6a2a2",
#               "#d9d9d9", "#bc80bd", "#ccebc5", "#ffed6f", "#5a3791", "#1d2d43", "#0068a0",
#               "#d9d9d9", "#bc80bd", "#ccebc5", "#ffed6f", "#f6c4d2", "#675974", "#bb1212",
#               "#d9d9d9", "#bc80bd", "#ccebc5", "#ffed6f", "#ffd03e", "#4f5573", "#327169",
#               "#3697dd", "#8dd3c7", "#ffffb3", "#bebada", "#fb8072", "#ea1717", "#66b4de",]

# fig = ff.create_choropleth(
#       fips=fips, values=cases, scope=scope,
#       colorscale=colorscale,round_legend_values=True,
#       simplify_county=0, simplify_state=0,
#       county_outline={'color': 'rgb(15,15,55)', 'width': 0.5},
#       state_outline={'width': 0.5},
#       legend_title='Cases Per Area',
#       title='California')
    
    
# iplot(fig, filename='Choropleth Map Creation')

# df_full


In [151]:
##https://plotly.com/python/county-choropleth/
##https://plotly.com/python/mapbox-county-choropleth/#mapbox-access-tokens-and-base-map-configuration

# geo_data_file = '/Users/Dave/covid-map-dataset-april-6.json'
# init_notebook_mode(connected = True)

# import plotly.express as px

# fig = px.choropleth_mapbox(df_full, geojson=geo_data_file, locations='FIPS', color='Cases',
#                            color_continuous_scale="Viridis",
#                            range_color=(0, 12),
#                            mapbox_style="carto-positron",
#                            zoom=8, center = {"lat": 34.0522, "lon": -118.2437},
#                            opacity=0.5,
#                            labels={'Cases':'Number of Cases'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

# fig.write_image("/Users/Dave/fignow.jpeg")

In [149]:
# #https://medium.com/@saidakbarp/interactive-map-visualization-with-folium-in-python-2e95544d8d9b
# import folium 

# # Initialize the map:
# # SoCal latitude and longitude values
# latitude = 34.0522
# longitude = -118.2437

# # create map and display it

# socal_map = folium.Map(location=[latitude, longitude], zoom_start=8)
# geo_data_file = "/Users/Dave/covid-map-dataset-april-6.json"

# # # Add the color for the chloropleth:
# socal_map.choropleth(
#  geo_data=geo_data_file,
#  name='choropleth',
#  data=df_full,
#  columns=['Area', 'Cases'],
#  key_on='FIPS',
#  fill_color='YlOrRd',
#  fill_opacity=0.7,
#  line_opacity=0.2,
#  legend_name='This Legend',
#     highlight=True
# )
# folium.LayerControl().add_to(socal_map)
# socal_map
